In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
from glob import glob
import json
import csv
import os

import cv2
import numpy as np

In [4]:
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [5]:
from keras.optimizers import Adam

In [42]:
from keras.layers import Lambda

In [6]:
from utils import read_samples
from utils import ImageGenerator

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
ROOT_DIR = os.path.join(os.getcwd(), '../')

In [9]:
DATA_DIR = os.path.join(ROOT_DIR, 'data')

In [10]:
samples = read_samples(DATA_DIR)

Loaded 13305 samples.


In [11]:
def append_path(line):
    line[0] = os.path.join(ROOT_DIR, line[0])
    line[1] = os.path.join(ROOT_DIR, line[1])
    line[2] = os.path.join(ROOT_DIR, line[2])
    return line

In [12]:
samples = [append_path(line) for line in samples]

In [40]:
train_set, test_set = train_test_split(samples, test_size=0.2)
train_set, valid_set = train_test_split(train_set, test_size=0.2)

In [55]:
_input = Input(shape=(80, 320, 3))
norm = Lambda(lambda x: x / 255 - 0.5)(_input)
conv1 = Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1), 
               padding='valid', activation='relu')(norm)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv1)
conv2 = Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), 
               padding='valid', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv2)
flatten = Flatten()(pool2)
fc1 = Dense(120, activation='linear')(flatten)
fc2 = Dense(84, activation='linear')(fc1)
output = Dense(1, activation='linear')(fc2)

In [56]:
train_iterator = ImageGenerator(train_set, batch_size=128, flip=True, gray=False)
valid_iterator = ImageGenerator(valid_set, batch_size=128, flip=True, gray=False)

In [57]:
adam = Adam(0.001)
model = Model(inputs=[_input], outputs=[output])
model.compile(loss='mse', optimizer=adam)

In [58]:
model.fit_generator(generator=train_iterator, epochs=10, 
                    validation_data=valid_iterator,
                    steps_per_epoch=len(train_iterator), 
                    validation_steps=len(valid_iterator))

Epoch 1/10
400/400 [==============================] - 125s - loss: 0.2385 - val_loss: 0.0226
Epoch 2/10
400/400 [==============================] - 124s - loss: 0.0213 - val_loss: 0.0246
Epoch 3/10
400/400 [==============================] - 124s - loss: 0.0088 - val_loss: 0.0201
Epoch 4/10
400/400 [==============================] - 124s - loss: 0.0050 - val_loss: 0.0295
Epoch 5/10
400/400 [==============================] - 124s - loss: 0.0030 - val_loss: 0.0278
Epoch 6/10
400/400 [==============================] - 124s - loss: 0.0032 - val_loss: 0.0317
Epoch 7/10
400/400 [==============================] - 124s - loss: 0.0024 - val_loss: 0.0310
Epoch 8/10
400/400 [==============================] - 124s - loss: 0.0018 - val_loss: 0.0291
Epoch 9/10
400/400 [==============================] - 124s - loss: 0.0012 - val_loss: 0.0238
Epoch 10/10
400/400 [==============================] - 125s - loss: 0.0011 - val_loss: 0.0238


In [59]:
model.save('lenet-128-20-flipped-col-001-norm.h5')

In [62]:
test_iterator = ImageGenerator(test_set, batch_size=128, gray=False)

In [63]:
results = model.evaluate_generator(generator=test_iterator, steps=len(test_iterator))

In [64]:
results

0.027762561326935178

In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 80, 320, 3)        0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 80, 320, 3)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 76, 316, 8)        608       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 38, 158, 8)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 34, 154, 16)       3216      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 77, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 20944)             0         
__________